In [3]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv("/Users/andreysorokin/Desktop/EBAY_20220201.csv", compression = 'zip')
data['dttm'] = data['dttm'].apply(pd.to_datetime) #изначально строка переводим в дату
data

,order_direction,order_action,price,quantity,traded_at_ts,traded_at_micro,dttm,session
0,bid,add,58.31,805,1643698800,264709,2022-02-01 10:00:00.264709+03:00,pre
1,bid,add,58.81,805,1643698800,264709,2022-02-01 10:00:00.264709+03:00,pre
2,bid,add,59.37,180,1643698800,264709,2022-02-01 10:00:00.264709+03:00,pre
3,ask,add,59.83,180,1643698800,264710,2022-02-01 10:00:00.264710+03:00,pre
4,ask,add,60.31,805,1643698800,264710,2022-02-01 10:00:00.264710+03:00,pre
...,...,...,...,...,...,...,...,...
291566,ask,delete,60.35,0,1643757482,853428,2022-02-02 02:18:02.853428+03:00,post
291567,ask,delete,60.48,0,1643757482,853428,2022-02-02 02:18:02.853428+03:00,post
291568,ask,delete,60.63,0,1643757482,853429,2022-02-02 02:18:02.853429+03:00,post
291569,ask,delete,60.90,0,1643757482,853429,2022-02-02 02:18:02.853429+03:00,post


In [8]:
data['dttm'].values

array(['2022-02-01T07:00:00.264709000', '2022-02-01T07:00:00.264709000',
       '2022-02-01T07:00:00.264709000', ...,
       '2022-02-01T23:18:02.853429000', '2022-02-01T23:18:02.853429000',
       '2022-02-01T23:18:02.853429000'], dtype='datetime64[ns]')

In [62]:
#data = pd.read_excel('data.xlsx')
#data['dttm'] = data['dttm'].apply(pd.to_datetime)
#data

In [67]:
# создаем промежуточный датафрейм, в который будем собирать инфу о бидах и асках
bid_ask_df = pd.DataFrame(columns = ['order_direction', 'price', 'quantity'])


for i in range(len(data)):
    # этот кусок проверяет, соответствует ли дата строки необходимому времени; если она больше, то цикл останавливается
    if data['dttm'].values[i]>= pd.to_datetime('2022-02-01 14:00:00.000000+03:00').tz_convert(None):
        break
    
    # собираем инфу о каждом событии в отдельные переменные
    order_direction = data['order_direction'].values[i]
    order_action = data['order_action'].values[i]
    price = data['price'].values[i]
    quantity = data['quantity'].values[i]
    
    
    if order_action == 'add':
        # если соблюдается условие, то в таблицу добавляется новая строчка ask|bid
        bid_ask_df.loc[i] = [order_direction, price, quantity]
    elif order_action == 'change':
        # если соблюдается условие, то меняется существующее количество лотов в имеющейся заявке
        bid_ask_df.loc[(( bid_ask_df['order_direction']==order_direction) &\
                       ( bid_ask_df['price']==price))\
                       ,'quantity'] = quantity
    elif order_action == 'delete':
        # если соблюдается условие, то из стакана удаляется заявка с такой ценой и таким order-direction
        bid_ask_df = bid_ask_df[(bid_ask_df['price']!=price )& (bid_ask_df['order_direction']!=order_direction)]

# сортируем получившуюся таблицу, чтобы получить стакан по убыванию цены
bid_ask_df = bid_ask_df.sort_values(by='price',ascending = False)

# создаем новый датафрейм, в котором сформируем стакан в том виде, который был представлен в задании
new_bid_ask_df = pd.DataFrame()
bid=[]
price=[]
ask = []
for i in range(len(bid_ask_df)):
    order_direction = bid_ask_df['order_direction'].values[i]
    if order_direction =='ask':
        bid.append('-')
        ask.append(bid_ask_df['quantity'].values[i])
    elif  order_direction =='bid':
        bid.append(bid_ask_df['quantity'].values[i])
        ask.append('-')
    price.append(bid_ask_df['price'].values[i])
new_bid_ask_df['bid'] = bid
new_bid_ask_df['price'] = price
new_bid_ask_df['ask'] = ask
new_bid_ask_df

,bid,price,ask
0,-,60.05,180
